In [1]:
pip install ratelimiter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import requests
from time import sleep
import datetime
from ratelimiter import RateLimiter
from google.colab import drive

In [3]:
airports = pd.read_csv("airports.csv")

france = airports[airports['CountryCode'] == "FR"].reset_index(drop=True)["AirportCode"]

european_codes = ["AL", "AD", "AT", "BY", "BE", "BA", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", 
                  "GR", "HU", "IS", "IE", "IT", "XK", "LV", "LI", "LT", "LU", "MK", "MT", "MD", "MC", "ME", 
                  "NL", "NO", "PL", "PT", "RO", "RU", "SM", "RS", "SK", "SI", "ES", "SE", "CH", "UA", "GB"]
europe = airports[airports['CountryCode'].isin(european_codes)].reset_index(drop=True)["AirportCode"]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#creation des listes
np.random.seed(123)

ousmane , abdoul , abdel = np.array_split(np.random.permutation(europe), 3)

In [12]:
import requests
from ratelimiter import RateLimiter
import pandas as pd

def get_flight_information_departures(origins, date, access_tokens):
    flights = []

    rate_limiter = RateLimiter(max_calls=4, period=1)  # 4 requêtes par seconde
    api_key_index = 0
    requests_count = 0

    for origin in origins:
        for hour in range(24):
            for minute in ["00", "30"]:
                with rate_limiter:
                    if requests_count >= 750:
                        print(f"Changement de clé API : utilisation de la clé {api_key_index + 1}") 
                        api_key_index += 1
                        requests_count = 0
                        if api_key_index >= len(access_tokens):
                            print("Toutes les clés API ont atteint la limite de requêtes.")
                            return pd.DataFrame.from_dict(flights).drop_duplicates().reset_index(drop=True)

                    time_str = f"{hour:02d}:{minute}"
                    datetime = f"{date}T{time_str}"
                    
                    authentification = {"Authorization": "Bearer " + access_tokens[api_key_index]}

                    try:
                        response = requests.get(
                            ("https://api.lufthansa.com/v1/operations/customerflightinformation/departures/{origin}/{datetime}"
                             "?limit=100&offset=0"
                            ).format(origin=origin, datetime=datetime),
                            headers=authentification).json()

                        for flight in response['FlightInformation']['Flights']['Flight']:
                            # Extraction des données du vol comme 
                            departure = flight.get('Departure', {})
                            arrival = flight.get('Arrival',{})
                            operatingCarrier = flight.get('OperatingCarrier')

                            row = {
                                'DpAirportCode': departure.get('AirportCode'),
                                'DpScheduledDate': departure.get('Scheduled', {}).get('Date'),
                                'DpScheduledTime': departure.get('Scheduled', {}).get('Time'),
                                'DpActualDate': departure.get('Actual', {}).get('Date'),
                                'DpActualTime': departure.get('Actual', {}).get('Time'),

                                'DpTerminalName': departure.get('Terminal', {}).get('Name'),
                                'DpTerminalGate': departure.get('Terminal', {}).get('Gate'),
                                'DpStatusCode': departure.get('Status', {}).get('Code'),
                                'StatusDescription': departure.get('Status', {}).get('Description'),
                                'AirlineID': operatingCarrier.get('AirlineID'),
                                'FlightNumber': operatingCarrier.get('FlightNumber'),
                                'ArrActualDate':arrival.get('Actual', {}).get('Date'),
                                'ArrActualTime':arrival.get('Actual', {}).get('Time'),
                                'ArrTerminalName':arrival.get('Terminal', {}).get('Name'),
                                'ArrTerminalGate':arrival.get('Terminal', {}).get('Gate'),
                                'ArrStatusCode':arrival.get('Status', {}).get('Code'),
                                'ArrStatusDescription':arrival.get('Status', {}).get('Description')
                            }

                            flights.append(row)

                        requests_count += 1

                    except Exception as e:
                        print(f"Error for origin {origin}, time {time_str}: {e}")

    flight_information_departures = pd.DataFrame.from_dict(flights).reset_index(drop = True)
    flight_information_departures = flight_information_departures.drop_duplicates().reset_index(drop = True)
    return flight_information_departures

access_tokens = ["aua2vxy7rx6tat5hauhd58fk", "u766bucht98baea8wf4dcque", 
                 "vkx6gd5av8jkk7y8mbe85t8z", "y89d4zjnjdwcy336q7h4snq8",
                 "s382tynhnrt5xkv62cjwfcea", "xsszwfc7w6h7ef8tespbm9d8",
                 "nsdyuxb4dwbkyphu95ynmbph", "zayjqwx2tpsuf52wbacw2qje"]
origins = ousmane
date = "2023-05-07"

flight_information_departures = get_flight_information_departures(origins, date, access_tokens)
flight_information_departures.to_csv("/content/drive/MyDrive/flight_information_departures.csv", index=False)
display(flight_information_departures)

Error for origin FRA, time 00:00: 'FlightInformation'
Error for origin FRA, time 00:30: 'FlightInformation'
Changement de clé API : utilisation de la clé 1
Error for origin FRA, time 07:30: [Errno Expecting value] <error>
  <category>Server</category>
  <text>APPL0014: IO Exception: Read timed out
java.net.SocketTimeoutException: Read timed out</text>
</error>
: 0
Error for origin FRA, time 09:00: 'FlightInformation'
Changement de clé API : utilisation de la clé 2
Error for origin FRA, time 19:00: 'FlightInformation'
Changement de clé API : utilisation de la clé 3
Error for origin FRA, time 21:00: 'FlightInformation'
Error for origin FRA, time 23:00: 'str' object has no attribute 'get'
Error for origin FRA, time 23:30: 'str' object has no attribute 'get'


,DpAirportCode,DpScheduledDate,DpScheduledTime,DpActualDate,DpActualTime,DpTerminalName,DpTerminalGate,DpStatusCode,StatusDescription,AirlineID,FlightNumber,ArrActualDate,ArrActualTime,ArrTerminalName,ArrTerminalGate,ArrStatusCode,ArrStatusDescription
0,FRA,2023-05-07,04:50,2023-05-07,05:15,1,A23,DP,Flight Departed,4Y,1204,2023-05-07,09:07,None,None,LD,Flight Landed
1,FRA,2023-05-07,04:50,2023-05-07,05:02,1,A16,DP,Flight Departed,LH,1506,2023-05-07,07:16,None,None,LD,Flight Landed
2,FRA,2023-05-07,04:55,2023-05-07,05:20,1,A20,DP,Flight Departed,4Y,1200,2023-05-07,09:16,None,None,LD,Flight Landed
3,FRA,2023-05-07,05:05,2023-05-07,05:13,1,A25,DP,Flight Departed,4Y,1206,2023-05-07,09:02,None,None,LD,Flight Landed
4,FRA,2023-05-07,05:25,2023-05-07,05:35,1,A18,DP,Flight Departed,4Y,1210,2023-05-07,08:54,None,None,LD,Flight Landed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,FRA,2023-05-07,22:15,2023-05-07,22:42,1,A22,DP,Flight Departed,LH,216,2023-05-07,23:39,None,None,LD,Flight Landed
528,FRA,2023-05-07,22:15,2023-05-07,22:22,1,Z66,DP,Flight Departed,LH,576,2023-05-08,10:14,None,None,LD,Flight Landed
529,FRA,2023-05-07,22:15,None,None,1,A18,NO,No Status,LH,1402,None,None,2,None,NO,No Status
530,FRA,2023-05-07,22:21,2023-05-07,22:40,Station,T06,DP,Flight Departed,2A,3496,2023-05-08,00:10,None,None,LD,Flight Landed


In [17]:
import requests
from ratelimiter import RateLimiter
import pandas as pd

def get_flight_information_arrivals(origins, date, access_tokens):
    flights = []

    rate_limiter = RateLimiter(max_calls=4, period=1)  # 4 requêtes par seconde
    api_key_index = 0
    requests_count = 0

    for origin in origins:
        for hour in range(24):
            for minute in ["00", "30"]:
                with rate_limiter:
                    if requests_count >= 750:
                        print(f"Changement de clé API : utilisation de la clé {api_key_index + 1}") 
                        api_key_index += 1
                        requests_count = 0
                        if api_key_index >= len(access_tokens):
                            print("Toutes les clés API ont atteint la limite de requêtes.")
                            return pd.DataFrame.from_dict(flights).drop_duplicates().reset_index(drop=True)

                    time_str = f"{hour:02d}:{minute}"
                    datetime = f"{date}T{time_str}"
                    
                    authentification = {"Authorization": "Bearer " + access_tokens[api_key_index]}

                    try:
                        response = requests.get(
                            ("https://api.lufthansa.com/v1/operations/customerflightinformation/arrivals/{origin}/{datetime}"
                             "?limit=100&offset=0"
                            ).format(origin=origin, datetime=datetime),
                            headers=authentification).json()

                        for flight in response['FlightInformation']['Flights']['Flight']:
                            # Extraction des données du vol comme 
                            arrival = flight.get('Arrival',{})
                            departure = flight.get('Departure', {})
                            operatingCarrier = flight.get('OperatingCarrier')

                            row = {
                                'ArrAirportCode': arrival.get('AirportCode'),
                                'ArrScheduledDate': arrival.get('Scheduled', {}).get('Date'),
                                'ArrScheduledTime': arrival.get('Scheduled', {}).get('Time'),
                                'ArrActualDate': arrival.get('Actual', {}).get('Date'),
                                'ArrActualTime': arrival.get('Actual', {}).get('Time'),

                                'ArrTerminalName': arrival.get('Terminal', {}).get('Name'),
                                'ArrTerminalGate': arrival.get('Terminal', {}).get('Gate'),
                                'ArrStatusCode': arrival.get('Status', {}).get('Code'),
                                'StatusDescription': arrival.get('Status', {}).get('Description'),
                                'AirlineID': operatingCarrier.get('AirlineID'),
                                'FlightNumber': operatingCarrier.get('FlightNumber'),
                                'DpActualDate':departure.get('Actual', {}).get('Date'),
                                'DpActualTime':departure.get('Actual', {}).get('Time'),
                                'DpTerminalName':departure.get('Terminal', {}).get('Name'),
                                'DpTerminalGate':departure.get('Terminal', {}).get('Gate'),
                                'DpStatusCode':departure.get('Status', {}).get('Code'),
                                'DpStatusDescription':departure.get('Status', {}).get('Description')

                            }
                            flights.append(row)

                        requests_count += 1

                    except Exception as e:
                        print(f"Error for origin {origin}, time {time_str}: {e}")

    flight_information_arrivals = pd.DataFrame.from_dict(flights).reset_index(drop = True)
    flight_information_arrivals = flight_information_arrivals.drop_duplicates().reset_index(drop = True)
    return flight_information_arrivals

access_tokens = ["aua2vxy7rx6tat5hauhd58fk", "u766bucht98baea8wf4dcque", 
                 "vkx6gd5av8jkk7y8mbe85t8z", "y89d4zjnjdwcy336q7h4snq8",
                 "s382tynhnrt5xkv62cjwfcea", "xsszwfc7w6h7ef8tespbm9d8",
                 "nsdyuxb4dwbkyphu95ynmbph", "zayjqwx2tpsuf52wbacw2qje"]
origins = ousmane
date = "2023-05-07"

flight_information_arrivals = get_flight_information_arrivals(origins, date, access_tokens)
flight_information_arrivals.to_csv("/content/drive/MyDrive/flight_information_arrivals.csv", index=False)
display(flight_information_arrivals)

Error for origin FRA, time 00:00: 'FlightInformation'
Error for origin FRA, time 00:30: 'FlightInformation'
Error for origin FRA, time 01:00: 'FlightInformation'
Error for origin FRA, time 04:00: 'FlightInformation'
Changement de clé API : utilisation de la clé 1
Error for origin FRA, time 07:30: 'FlightInformation'
Changement de clé API : utilisation de la clé 2
Changement de clé API : utilisation de la clé 3
Changement de clé API : utilisation de la clé 4
Error for origin FRA, time 17:00: [Errno Expecting value] <error>
  <category>Server</category>
  <text>APPL0014: IO Exception: Read timed out
java.net.SocketTimeoutException: Read timed out</text>
</error>
: 0
Changement de clé API : utilisation de la clé 5
Error for origin FRA, time 19:30: 'FlightInformation'
Changement de clé API : utilisation de la clé 6
Error for origin FRA, time 23:30: 'FlightInformation'


,ArrAirportCode,ArrScheduledDate,ArrScheduledTime,ArrActualDate,ArrActualTime,ArrTerminalName,ArrTerminalGate,ArrStatusCode,StatusDescription,AirlineID,FlightNumber,DpActualDate,DpActualTime,DpTerminalName,DpTerminalGate,DpStatusCode,DpStatusDescription
0,FRA,2023-05-07,05:25,2023-05-07,05:10,1,C05A,LD,Flight Landed,LH,1327,2023-05-07,01:47,M,059,DP,Flight Departed
1,FRA,2023-05-07,05:26,2023-05-07,05:24,TN,None,LD,Flight Landed,2A,3599,2023-05-07,04:21,None,T05,DP,Flight Departed
2,FRA,2023-05-07,05:30,2023-05-07,05:38,1,Z62,LD,Flight Landed,LH,417,2023-05-06,15:29,None,B45,DP,Flight Departed
3,FRA,2023-05-07,05:30,2023-05-07,05:30,1,Z69A,LD,Flight Landed,LH,577,2023-05-06,17:47,None,A06,DP,Flight Departed
4,FRA,2023-05-07,05:35,2023-05-07,05:07,1,C14A,LD,Flight Landed,4Y,143,2023-05-06,18:46,None,None,DP,Flight Departed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,FRA,2023-05-07,21:55,None,None,1,A11,NO,No Status,LH,203,None,None,1,None,NO,No Status
527,FRA,2023-05-07,22:00,2023-05-07,22:00,1,A20,LD,Flight Landed,LH,123,2023-05-07,21:01,2,G30,DP,Flight Departed
528,FRA,2023-05-07,22:05,2023-05-07,22:42,1,B20,LD,Flight Landed,LH,919,2023-05-07,20:03,2,B37,DP,Flight Departed
529,FRA,2023-05-07,22:30,2023-05-07,22:42,1,A13,LD,Flight Landed,4Y,307,2023-05-07,16:49,None,None,DP,Flight Departed
